In [4]:
!pip install -U efficientnet==0.0.4
# Efficient Netを実行するには以下のtensorflowとkerasが必要．
# しかしcudaやcuDNNのバージョンがあってないのでkaggleのnotebookでは難しい．

#!pip install --upgrade tensorflow-gpu==1.12.0
#!pip install --upgrade keras==2.2.4

  Created wheel for efficientnet: filename=efficientnet-0.0.4-cp36-none-any.whl size=14288 sha256=a9ce648896679f84607f20be72076d1832a481e56a5877e53186006712e11b85
  Stored in directory: /root/.cache/pip/wheels/5c/34/68/a611a699a28239e964ccf144c0e767cdb5439fee82ec5de6e0
Successfully built efficientnet


In [5]:
# Dataloder ： utils/loder.pyに記述するべき内容
import numpy as np
from math import ceil
from scipy import ndimage
from sklearn.model_selection import train_test_split

from tensorflow.keras import utils
import keras
#from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.datasets.mnist import load_data

class Dataloder(object):
    def __init__(self):
        pass
    
    def get_data(self, channel):
        # load MNIST data
        (x_train, y_train), (x_test, y_test) = load_data()
        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.175)

        self.x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')/255
        self.x_valid = x_valid.reshape(x_valid.shape[0], 28, 28, 1).astype('float32')/255
        self.x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')/255

        if channel == 3:
            self.x_train = np.tile(x_train, 3)
            self.x_valid = np.tile(x_valid, 3)
            self.x_test = np.tile(x_test, 3)

        # convert one-hot vector
        self.y_train = keras.utils.to_categorical(y_train, 10)
        self.y_valid = keras.utils.to_categorical(y_valid, 10)
        self.y_test = keras.utils.to_categorical(y_test, 10)
    
        pass

In [68]:
# モジュールの構造：uitils.modules.pyに記述
from efficientnet import EfficientNetB0
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

def bottleneck(x, n_classes):
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.25)(x)
    x = Dense(n_classes, activation='sigmoid')(x)
    return x

def get_model(input_shape, n_classes):
    print("input_shape : {}".format(input_shape))
    # EfficientNet使う時
#     effnet_instance = EfficientNetB0(input_shape=input_shape[1:], weights='imagenet', include_top=False)
#     x = effnet_instance.output()

    # EfficientNet使わない時
    input_img = Input(input_shape[1:])
    x = input_img
    print("input" + str(x))
    
    x = bottleneck(x, n_classes)
    
    #model = Model(inputs=effnet_instance.input, outputs=x)
    model = Model(inputs=input_img, outputs=x)
    
    return model

In [78]:
# model.compileとmodel.fit : solver.pyに記載
from keras.optimizers import RMSprop
from keras.callbacks import Callback, CSVLogger

class Solver(object):
    def __init__(self, dataloder_instance, batch_size, n_epochs):
        self.data_loader = dataloder_instance
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.n_classes = 10
        print("n_classes : {}".format(self.n_classes))
        
        self.model = get_model(input_shape = self.data_loader.x_train.shape, n_classes = self.n_classes)
        
        # ファインチューニングする場合は専用のAdamオプティマイザを使う
        # optimizer = get_adam_for_fine_tuning(lr=1e-3, decay=1e-5, multiplier=0.01, model=model)
        
        self.model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),
                  metrics=['accuracy'])
    
    def train(self):
        # callbackを定義する場合はここに記述
        self.model.fit(self.data_loader.x_train, self.data_loader.y_train, batch_size=self.batch_size, \
                       epochs=self.n_epochs, verbose=1, validation_data=(self.data_loader.x_valid, self.data_loader.y_valid))

In [80]:
dataloder_ins = Dataloder()
dataloder_ins.get_data(channel = 1)

solver_ins = Solver(dataloder_ins, batch_size=10, n_epochs=1)
solver_ins.train()

n_classes : 10
input_shape : (49500, 28, 28, 1)
inputTensor("input_12:0", shape=(None, 28, 28, 1), dtype=float32)
Train on 49500 samples, validate on 10500 samples
Epoch 1/1
49500/49500 [==============================] - 16s 328us/step - loss: 2.1522 - accuracy: 0.2029 - val_loss: 2.0750 - val_accuracy: 0.2326


In [ ]:
import os, random, cv2
import numpy as np
from albumentations import Compose
from albumentations.augmentations.transforms import HorizontalFlip, Normalize

class DataLoader(object):
    def __init__(self, data_folder, batch_size, input_shape, do_augmentation, gray_scale=False):
        self._file_paths = []
        self._annotations = []
        folders = os.listdir(data_folder)
        folders.sort()
        # 画像のパスとクラスIDを取得する
        for class_id, class_name in enumerate(folders):
            folder_path = data_folder + os.sep + class_name
            file_paths = [folder_path + os.sep + fn for fn in os.listdir(folder_path + os.sep)]
            self._file_paths += file_paths
            self._annotations += [class_id]*len(file_paths)
        # クラス数
        self._class_count = class_id + 1
        self._batch_size = batch_size
        self._input_shape = input_shape
        self._gray_scale = gray_scale
        if len(self._file_paths)%self._batch_size == 0:
            self.iterations = len(self._file_paths) // self._batch_size
            self._has_extra_data = False
        else:
            self.iterations = len(self._file_paths) // self._batch_size + 1
            self._has_extra_data = True
        self._compose = self._define_augment(input_shape, do_augmentation)

    def _define_augment(self, input_shape, do_augmentation):
        # mean, std, max_pixel_valueは適宜変更してください
        mean = (0.485*255, 0.456*255, 0.406*255)
        std = (0.229*255, 0.224*255, 0.225*255)
        normalize = Normalize(mean=mean, std=std, max_pixel_value=1)
        # データ拡張内容は適宜変更してください
        if do_augmentation:
            return Compose([normalize, HorizontalFlip(p=0.5)])
        else:
            return Compose([normalize])

    def get_data_loader(self):
        while True:
            file_paths, annotations = self._shuffle(self._file_paths, self._annotations)
            for iteration in range(self.iterations):
                if iteration == self.iterations - 1 and self._has_extra_data:
                    shape = (len(file_paths)%self._batch_size, self._input_shape[0],
                             self._input_shape[1], self._input_shape[2])
                else:
                    shape = (self._batch_size, self._input_shape[0], self._input_shape[1], self._input_shape[2])
                # バッチサイズ分のデータを取得する
                X = np.zeros(shape, dtype=np.float32)
                y = np.zeros((shape[0], self._class_count), dtype=np.float32)
                for i in range(X.shape[0]):
                    index = self._batch_size*iteration + i
                    if self._gray_scale:
                        image = cv2.imread(file_paths[index], cv2.IMREAD_GRAYSCALE)
                        image = image[:,:,np.newaxis]
                    else:
                        image = cv2.imread(file_paths[index])
                    image = cv2.resize(image, (self._input_shape[1], self._input_shape[0]))
                    image = image.astype(np.float32)
                    # データ拡張を実行する
                    X[i] = self._augment(image)
                    y[i, annotations[index]] = 1
                yield X, y

    def _shuffle(self, x, y):
        p = list(zip(x, y))
        random.shuffle(p)
        return zip(*p)

    def _augment(self, image):
        dict = {'image': image}
        augmented = self._compose(**dict)
        return augmented['image']

In [ ]:
from keras.legacy import interfaces
import keras.backend as K
from keras.optimizers import Optimizer

class Adam_lr_mult(Optimizer):
    """Adam optimizer.
    Adam optimizer, with learning rate multipliers built on Keras implementation
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
    # References
        - [Adam - A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        
    AUTHOR: Erik Brorson
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False,
                 multipliers=None, debug_verbose=False,**kwargs):
        super(Adam_lr_mult, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad
        self.multipliers = multipliers
        self.debug_verbose = debug_verbose

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):

            # Learning rate multipliers
            if self.multipliers:
                multiplier = [mult for mult in self.multipliers if mult in p.name]
            else:
                multiplier = None
            if multiplier:
                new_lr_t = lr_t * self.multipliers[multiplier[0]]
                if self.debug_verbose:
                    print('Setting {} to learning rate {}'.format(multiplier[0], new_lr_t))
                    print(K.get_value(new_lr_t))
            else:
                new_lr_t = lr_t
                if self.debug_verbose:
                    print('No change in learning rate {}'.format(p.name))
                    print(K.get_value(new_lr_t))
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - new_lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - new_lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad,
                  'multipliers':self.multipliers}
        base_config = super(Adam_lr_mult, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def get_adam_for_fine_tuning(lr, decay, multiplier, model):
    lr_multiplier = {}
    # 自分が引っ付けたレイヤーの学習係数は1、学習済みの部分は小さな値を設定する
    for layer in model.layers:
        if 'dense' in layer.name:
            lr_multiplier[layer.name] = 1.0
        else:
            lr_multiplier[layer.name] = multiplier
    return Adam_lr_mult(lr=lr, decay=decay, multipliers=lr_multiplier)

In [ ]:
class ClsTestDataset(Dataset):

    def __init__(self, num_samples=1, model='densenet161'):
        
        self.num_samples = num_samples
        self.images = np.zeros([num_samples, WIDTH * HEIGHT], dtype=np.uint8)
    
        self.aug = albumentations.Compose([
            albumentations.Resize(int(HEIGHT), int(WIDTH), always_apply=True),
            albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], always_apply=True)
        ])
    
        img_id = 0

        for i in tqdm(range(4)):
            datafile = INPUT_PATH + '/test_image_data_{}.parquet'.format(i)
            parq = pq.read_pandas(datafile, columns=[str(x) for x in range(32332)]).to_pandas()
            parq =  parq.iloc[:, :].values.reshape(-1, HEIGHT, WIDTH).astype(np.uint8)
            
            for idx, image in enumerate(parq):
                self.images[img_id, ...] = image.reshape(-1).astype(np.uint8)
                img_id = img_id + 1
                
        del parq

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        img = self.images[idx]
        img = img.reshape(HEIGHT, WIDTH).astype(float)
        img = Image.fromarray(img).convert("RGB")
        img = self.aug(image=np.array(img))["image"]
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)
        img = torch.tensor(img, dtype=torch.float)

        return {
            "image": img,
            "image_id": idx
        }